In [1]:
from datasets import load_dataset
import datasets
import numpy as np
import librosa
import pandas as pd
import os
import soundfile as sf

In [2]:
sentences = pd.read_json("/Users/Stefan/Desktop/python tests/sentences/sentences_ch_de_transcribed.json")
sentences.set_index("id", inplace=True)
sentences.drop(["de","thema","code_switching"], axis=1, inplace= True)
sentences.head()

,ch_sg,ch_be,ch_gr,ch_zh,ch_vs,ch_bs,ch_ag,ch_lu
id,,,,,,,,
0,"Zur Ziit isch er in ""Parasite"", en Siegerfilm ...","Momentan ischer in ""Parasite"" z gseh, em Siege...","Derziit isch er in ""Parasite"", am Siegerfilm v...","Momentan chamer ihn in ""Parasite"", em Siegerfi...","Derzit ischer in ""Parasite"", dum Siegerfilm va...","Zur Ziit isch är in ""Parasite"", em Siigerfilm ...","Em Moment escher en ""OParasite"", de Siegerfelm...","Momentan escher en ""Parasite"", em Sigerfelm vo..."
1,dBoeing vom Typ sibedrüsibe achthundert NG seg...,D Boeing vom Typ sibä drü sibä achthundert NG ...,D Boeing vum Typ siebahundertsiebanädrissig ac...,D Boeing vom Typ sibedrüsibe-achthundert NG se...,Die Boeing vam Typ sibudriisibu-achthunert NG ...,D Boeing vom Typ siebedreisibe-achthundert NG ...,D'Boeing vom Typ sebehondertsebenedrissg-achth...,De Boeing vom Typ sebe drü sebe-achthondert NG...
2,Etz isch dPflegefachfrau sit Monate arbetsunfä...,Iz isch d Pflegefachfrou sit Monatä arbeitsunf...,Jetzt isch d Pflegefachfrau sit Mönet arbetsun...,Jetzt isch d Pflägefachfrau sit Monate arbetsu...,Jetz isch die Pflegfachfröi sit Monati arbeits...,Jetzt isch d Pflegefachfrau siit Monate arbets...,Jetzt esch d'Pflegefachfrau set Monate arbeits...,Jetz esch die Pflegefachfrau setem Mäntig arbe...
3,De isch au im April wieder zueverlässig uf sim...,Dä isch o im Apriu zueverlässig uf sim heissä ...,De isch im April au wieder zuaverlässig uf sin...,De isch au im April wider zueverlässig uf sim ...,Der isch öi im April wieder züeverlässig uf sc...,Dä isch au im April wieder zueverlässig uf sim...,De esch au em April weder zueverlässig of sim ...,Dä esch au em April weder zueverlässig uf sim ...
4,"""dParlamentarier werded sich bestimmt au uf di...","""D Parlamentarier wärdä sech beschtimmt o uf d...",""" D Parlamentarien werden sich sicher au uf di...","""D Parlamentarier beziehnd sich sicher au uf d...","""Die Parlamentarier wärdunt schich sicher öi u...","""D Parlamentarier wärde sich bstimmt au uf die...","""D'Parlamentarier wärde sech bestemmt au of di...","""D Parlamäntarier wärded sech secher au uf di ..."


In [3]:
sentences.describe()

,ch_sg,ch_be,ch_gr,ch_zh,ch_vs,ch_bs,ch_ag,ch_lu
count,2752,2700,2749,4065,2753,2713,2748,2715
unique,2750,2700,2748,4060,2749,2709,2742,2711
top,Vor allem am Samschtig gits i de Bärge lokali ...,"Es längt, wedes grob skizziersch.",Vor allem am Samstig gits in da Berga lokali R...,Wider mal isches de Jüngsti gsi wo am Esstisch...,"Das cha nit sii, ihm isch nu nie eppis üsgleert.",Scho vo wiitem hani könne erkenne das är sini ...,"Wederemol esch er de Jöngsti gsi, wo am Ässtes...","Das cha ned sie, ehm esch nonie öppis usgläärt."
freq,2,1,2,2,2,2,2,2


In [26]:
#define variables where data is stored
#take into account, that the online path is different from local storage

path = "./full_audio/data/" #local data path realtive to notebook
csv_path = "./csv/" #export path realtive to notebook
colab_path = "./full_audio/" #path on online notebook
toy_path = "./toy_audio/"

data=[]
target_sr = 16000 #target sample rate
train_size = 0.95 #train test split size

#iterate through dataset file structure to get information about the files stored and resample audio
#this takes a while: Approx 1 min / 1000 files
for root, directories, files in os.walk(path, topdown=False): 
    files = [f for f in files if not f[0] == '.' and not f[-1] == "l"] #make sure not to get hidden files or files ending with l (had a .pkl test in there at some point)
    for name in files:
        
        folder = os.path.join(root) #get folder name
        filename = os.path.join(name) #get filename
        path_name = os.path.join(root,name) #get path name for online use
        raw_audio, sample_rate = sf.read(str(os.path.join(root,name)),dtype="float32") #read audio and get sample rate for resampling
        
        if sample_rate != 16000:
            raw_audio_16K = librosa.resample(raw_audio, sample_rate, target_sr)  #resample audio
            sf.write(str(os.path.join(root,name)), raw_audio_16K, target_sr) #write file in place
            
        data.append([folder,filename,path_name,target_sr]) #append to list in praparation for data frame



df=pd.DataFrame(data, columns=["folder","filename","path_from_data_dir","sampling_rate"]) #create dataframe for audio
df.sort_values(by="filename", inplace = True) #sort dataframe to line up with text

columns_list = sentences.columns.values.tolist() #reshaping dataframe
columns_list.sort() #sort list to line up with audio
folder_list = df["folder"].unique().tolist() #unique folder list
folder_list.sort() #sort folder list to line up with audio and text
filenames = df["filename"].tolist() #get sorted filenames to list
path_from_data_dir = df["path_from_data_dir"].sort_values().tolist() #get sorted paths to audio
sample_rate = df["sampling_rate"].tolist() #to make sure, include sample rate

text = []
for i in columns_list:
    text.append(sentences[i].dropna().values.tolist()) #iterate through columns to get sentences maybe reshape would be better

text = [item for sublist in text for item in sublist] #complicated way to reshape as each column creates a sublist with the sentences in it   

#create train and test set in dataframe
train_or_test = [] 
for i in np.random.rand(len(df)) < train_size: #random generator with trining split
    if i > train_size:
        train_or_test.append("TRAIN")
    else:
        train_or_test.append("TEST")

#make sure everything has the same length
print(len(train_or_test)) 
print(len(filenames))
print(len(path_from_data_dir))
print(len(text))


#create dataframe to use
column_dict = {
               "audio_path":colab_path + df["filename"], 
               "text":text, 
               "train_or_test":train_or_test,
               }

zusammen = pd.DataFrame(data=column_dict) 
zusammen.reset_index(drop=True)
zusammen.head()

23195
23195
23195
23195


,audio_path,text,train_or_test
15563,./full_audio/ch_ag_0000.wav,"Em Moment escher en ""OParasite"", de Siegerfelm...",TRAIN
15619,./full_audio/ch_ag_0001.wav,D'Boeing vom Typ sebehondertsebenedrissg-achth...,TRAIN
15448,./full_audio/ch_ag_0002.wav,Jetzt esch d'Pflegefachfrau set Monate arbeits...,TRAIN
15401,./full_audio/ch_ag_0003.wav,De esch au em April weder zueverlässig of sim ...,TRAIN
15137,./full_audio/ch_ag_0004.wav,"""D'Parlamentarier wärde sech bestemmt au of di...",TRAIN


In [17]:
df.head()

,folder,filename,path_from_data_dir,sampling_rate
15563,./full_audio/data/ag,ch_ag_0000.wav,./full_audio/data/ag/ch_ag_0000.wav,16000
15619,./full_audio/data/ag,ch_ag_0001.wav,./full_audio/data/ag/ch_ag_0001.wav,16000
15448,./full_audio/data/ag,ch_ag_0002.wav,./full_audio/data/ag/ch_ag_0002.wav,16000
15401,./full_audio/data/ag,ch_ag_0003.wav,./full_audio/data/ag/ch_ag_0003.wav,16000
15137,./full_audio/data/ag,ch_ag_0004.wav,./full_audio/data/ag/ch_ag_0004.wav,16000


In [22]:
#Copy file to new location as per dataset path requirements
from shutil import copyfile
full_audio_folder = "./full_audio_folder/"
for file in df["path_from_data_dir"]: 
    src_file = file
    dest_file = full_audio_folder + file[21:] #be lazy
    copyfile(src_file, dest_file)
print("Files copied")

Files copied


In [28]:
train_prepared = zusammen.loc[zusammen["train_or_test"] == "TRAIN",:] #create train set
test_prepared = zusammen.loc[zusammen["train_or_test"] == "TEST",:] #create test set
train_prepared.to_csv("train_data_full.csv")#write csv file of training set
test_prepared.to_csv("test_data_full.csv") #write csv file of test seet

In [27]:
len(zusammen.loc[zusammen["train_or_test"] == "TEST",:])

1175